In [59]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [54]:
import pandas as pd
from pymongo import MongoClient
import certifi
import os
from pathlib import Path
import sys
import requests
import numpy as np

In [3]:
path_ = Path(sys.path[0])
path_to_repo = str(path_.parent.parent.parent.absolute()) #здесь можно указать путь до папки, где лежит репозиторий
path = path_to_repo + '/recommendations/data/raw'

with open(path_to_repo+'/mongodb_pass.txt', 'r') as file:
    path2 = file.read()

path2 = path2.replace('\n','')

In [23]:
client = MongoClient(path2, tlsCAFile=certifi.where())
current_db = client['spin_services']

heats = pd.DataFrame(current_db['cs_cart_heats'].find())

In [13]:
r = requests.get('https://spin4spin.com/index.php?dispatch=stat.get_ab_recs&access_key=Qh7KXPMDbcNpqYssMw=pf$xhhggw!8')
groups = r.json()
groups_df = pd.DataFrame.from_dict(groups['recs']).T.set_index('ym_id')

groups_df.index = groups_df.index.str.strip()

In [40]:
merged = heats.loc[
    heats.current_url.str.contains('recommend', na=False)].groupby(
    'ym_client_id')._id.count().to_frame().join(groups_df)

In [45]:
merged = merged.loc[~merged.value.isna()]

In [51]:
merged_recs,merged_categs = merged.loc[merged.value == '1'], merged.loc[merged.value == '0']

In [60]:
merged.value = merged.value.astype(int)

/Users/kuznetsovnikita/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [62]:
model = smf.ols('_id ~ value', data = merged)

In [65]:
results = model.fit()

In [68]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    _id   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     3.519
Date:                Mon, 20 Jun 2022   Prob (F-statistic):             0.0626
Time:                        18:45:37   Log-Likelihood:                -338.02
No. Observations:                 155   AIC:                             680.0
Df Residuals:                     153   BIC:                             686.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.5211      0.256      9.853      0.000       2.016       3.027
value         -0.6521      0.348     -1.876      0.063      -1.339       0.035
==============================================================================
Omnibus:                      111.929   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              741.226
Skew:                           2.730   Prob(JB):                    1.11e-161
Kurtosis:                      12.217   Cond. No.                         2.73
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [69]:
merged_categs._id.mean(), merged_recs._id.mean()

(2.5211267605633805, 1.869047619047619)

In [ ]:
1625481536100736290 - хром
1654844530958233689 - сафари

In [ ]:
# сделать коллекцию с пользователями: 
# - кол-во заходов (визитов)
# - кол-во переходов по страницам
# - среднее время посещения
# - кол-во переходов 